In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-09 14:20:02--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  36.0MB/s    in 45s     

2021-01-09 14:20:48 (40.6 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-09 14:20:48--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [ ]:
!cp ../drive/MyDrive/slt/early-fusion-v3/*.py signjoey/

In [ ]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [ ]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [ ]:
body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:47<00:00, 149.83it/s]


In [ ]:
for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami2.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:33<00:00, 75.58it/s]


In [ ]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(3)]

list_doc['data']['feature_size'] = [1024, 417, 1024]
list_doc['training']['recognition_loss_weight'] = 20

list_doc["model"]["embeddings"] = []
list_doc["model"]["embeddings"].append(list_doc["model"]["encoder"]["embeddings"].copy())
for i in range(2):
  list_doc["model"]["encoder"]["embeddings"]["embedding_dim"] = 256
  list_doc["model"]["embeddings"].append(list_doc["model"]["encoder"]["embeddings"].copy())

list_doc["model"]["encoder"]["embeddings"]["embedding_dim"]   = 1024
list_doc["model"]["encoder"]["hidden_size"] = 1024
list_doc["model"]["decoder"]["embeddings"]["embedding_dim"]   = 1024
list_doc["model"]["decoder"]["hidden_size"] = 1024

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
%cd slt

/content/slt


In [ ]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
2021-01-09 15:07:52,054 	Text Alignment  :	D   S             S        S      S    S         S          S            S     S       
2021-01-09 15:07:52,054 ========================================================================================================================
2021-01-09 15:08:17,274 [Epoch: 014 Step: 00003100] Batch Recognition Loss:  28.616253 => Gls Tokens per Sec:      981 || Batch Translation Loss:  23.277254 => Txt Tokens per Sec:     2007 || Lr: 0.001000
2021-01-09 15:08:51,511 Hooray! New best validation result [eval_metric]!
2021-01-09 15:08:51,512 Saving new checkpoint.
2021-01-09 15:08:54,231 Validation result at epoch  14, step     3100: duration: 36.9571s
	Recognition Beam Size: 1	Translation Beam Size: 1	Translation Beam Alpha: -1
	Recognition Loss: 704.31140	Translation Loss: 20131.35156	PPL: 12.96032
	Eval Metric: BLEU
	WER 64.34	(DEL: 61.04,	INS: 0.35,	SUB: 2.96)
	BLEU-4 15.78	(BLEU-1: 41.06,	BLEU-2: 27.

In [9]:
!python -m signjoey test configs/sign.yaml

2021-01-09 16:41:01.663602: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-09 16:41:09.805990: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-09 16:41:09.828992: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-09 16:41:09.888411: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-09 16:41:09.889416: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-01-09 16:41:09.889460: I tensorflow/stream_executor/platform/def

In [ ]:
33.33 & 19.75 & 33.74 & 18.96